In [1]:
from dataclasses import dataclass

from dotenv import load_dotenv
import logfire
from pydantic_ai import Agent, RunContext, ModelRetry
from pydantic_ai.models import ModelResponse
from pydantic_ai.messages import ToolCallPart

from pydanticai_examples.config import get_qdrant_config, get_openai_config, get_todoist_config, get_logfire_config
from pydanticai_examples.vector_store import VectorStore
from pydanticai_examples.todoist import TodoistClient
from pydanticai_examples.utils import get_root

In [2]:
load_dotenv(get_root() / ".env")

True

In [3]:
#Configure logfire
logfire_config = get_logfire_config()
logfire.configure(token=logfire_config.logfire_token)
logfire.instrument_openai()

In [4]:
@dataclass
class RecipeAgentDeps:
    """Class to represent the dependencies for the Recipe Agent"""
    vector_store: VectorStore
    todoist_client: TodoistClient

In [5]:
@dataclass
class ChosenRecipe:
    """Class to represent a chosen recipe"""
    name: str
    description: str
    ingredients: list[str]
    instructions: str

    def __str__(self):
        return f"{self.name}: {self.description}\nIngredients: {', '.join(self.ingredients)}\nInstructions: {self.instructions}"

In [6]:
cooking_agent = Agent[RecipeAgentDeps, ChosenRecipe](
    model='openai:gpt-4-turbo',
    deps_type=RecipeAgentDeps,
    result_type=ChosenRecipe,
    instrument=True,
    system_prompt="""You are a professional dietitian. For a given list of ingredients that a user has in their stash, you should suggest a recipe that uses those ingredients.
You should only use recipes that are contained in the vector database.

If the list of ingredients provided by the user is missing some of the ingredients from the chosen recipe, create a Todoist task to buy a missing ingredients.
The task title should be "Buy missing ingredients for {recipe_name}".
The task details should include the list of missing ingredients.
    """
)

In [7]:

@cooking_agent.tool
async def search_for_recipes(ctx: RunContext[RecipeAgentDeps], ingredients: list[str]) -> list[dict]:
    """Search the vector store for cooking recipes
    including the ingredients provided.

    Args:
        ingredients (list[str]): The list of ingredients the resulting
            recipes should include.

    Returns:
        list[str]: List of recipes found for a given query.
    """
    # Embed the query
    ingredients_str = ", ".join(ingredients)
    return await ctx.deps.vector_store.search(ingredients_str)

In [8]:
@cooking_agent.tool
async def add_task_to_todoist(ctx: RunContext[RecipeAgentDeps], task_title: str, task_details: str) -> list[dict]:
    """Add a task to Todoist todo app.

    Args:
        task_title (str): The title of the task.
        task_details (str): The details of the task.

    Returns:
        list[dict]: The created task.
    """
    return await ctx.deps.todoist_client.add_task(task_title, task_details)

In [9]:
@cooking_agent.result_validator
async def check_if_task_added(ctx: RunContext[RecipeAgentDeps], result: ChosenRecipe) -> ChosenRecipe:
    """Validator to check if a task was added to Todoist.

    Args:
        ctx (RunContext[RecipeAgentDeps]): The context of the run.
        result (ChosenRecipe): The result of the agent.

    Returns:
        ChosenRecipe: The result of the agent.

    Raises:
        ModelRetry: If the agent did not create a task to buy missing ingredients.
    """
    todoist_used = False
    for msg in ctx.messages:
        if isinstance(msg, ModelResponse):
            for part in msg.parts:
                if isinstance(part, ToolCallPart):
                    if part.tool_name == "add_task_to_todoist":
                        todoist_used = True
                        break
    if not todoist_used:
        raise ModelRetry("The agent did not create a task to buy missing ingredients.")
    return result


In [10]:
# Instantiate the vector store
qdrant_config = get_qdrant_config()
openai_config = get_openai_config()
vector_store = VectorStore(
    url=qdrant_config.qdrant_url,
    port=qdrant_config.qdrant_port,
    collection_name=qdrant_config.qdrant_index_name,
    openai_api_key=openai_config.openai_api_key,
)

# Instantiate the Todoist client
todoist_config = get_todoist_config()
todoist_client = TodoistClient(
    api_key=todoist_config.todoist_api_key,
    project=todoist_config.todoist_project,
)

# Gather dependencies into a single object
deps = RecipeAgentDeps(
    vector_store=vector_store,
    todoist_client=todoist_client,
)

In [11]:
result = await cooking_agent.run("I have only milk and wheat", deps=deps)

19:20:50.610 cooking_agent run
19:20:50.611   preparing model request params
19:20:50.612   chat gpt-4-turbo
19:20:50.622     Chat Completion with 'gpt-4-turbo' [LLM]
19:20:52.507   running tools: search_for_recipes
19:20:52.508     Embedding Creation with 'text-embedding-3-small' [LLM]
19:20:52.921   preparing model request params
19:20:52.921   chat gpt-4-turbo
19:20:52.925     Chat Completion with 'gpt-4-turbo' [LLM]
19:20:59.310   preparing model request params
19:20:59.311   chat gpt-4-turbo
19:20:59.317     Chat Completion with 'gpt-4-turbo' [LLM]


In [12]:
result.data

ChosenRecipe(name='French Toast', description='Simple french toast recipe that can be enjoyed with various toppings.', ingredients=['milk', 'butter', 'bread', 'sugar', 'cinnamon', 'eggs'], instructions='1. Beat the eggs and milk in a bowl.\n2. Add the sugar and cinnamon.\n3. Take one slice of bread and place it in the egg mixture, soak it on both sides for 30 seconds.\n4. Melt the butter in the pan, making sure it doesn’t begin to brown.\n5. Add the bread and fry for around 2 minutes on each side.\nOptional: Leave out the sugar and have with honey, jam or maple or golden syrup.')

In [13]:
result.usage()

Usage(requests=2, request_tokens=1339, response_tokens=213, total_tokens=1552, details={'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0, 'cached_tokens': 0})